In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import yt

import sys,os
sys.path.append('../../')
sys.path.append('../../pyathena/')
import pyathena as pa
import tigradpy as tp
print(tp.__all__)
u = tp.units()

['add_fields', 'mass_to_lum', 'read_athinput', 'read_hst', 'read_starpar_vtk', 'read_zprof', 'read_zprof_all', 'units', 'yt_multipanel']


In [5]:
model = 'R8_8pc_newacc'
num = 1
datadir = '/tigress/jk11/radps_postproc/R8_8pc_newacc_zmax3584_irunaway0_nproc252/'
fname = os.path.join(datadir, '{0:s}.{1:04d}.vtk'.format(model,num))
ds = yt.load(fname, units_override=u.units_override)
tp.add_fields(ds)

yt : [INFO     ] 2018-10-09 01:32:54,954 Temporarily setting domain_right_edge = -domain_left_edge. This will be corrected automatically if it is not the case.
yt : [WARNING  ] 2018-10-09 01:32:54,956 Overriding code units: Use this option only if you know that the dataset doesn't define the units correctly or at all.
yt : [INFO     ] 2018-10-09 01:32:54,957 Overriding length_unit: 1 pc.
yt : [INFO     ] 2018-10-09 01:32:54,959 Overriding time_unit: 0.977792 Myr.
yt : [INFO     ] 2018-10-09 01:32:54,960 Overriding mass_unit: 0.0352939 Msun.
yt : [INFO     ] 2018-10-09 01:32:54,975 Parameters: current_time              = 250.0011
yt : [INFO     ] 2018-10-09 01:32:54,977 Parameters: domain_dimensions         = [128 128 896]
yt : [INFO     ] 2018-10-09 01:32:54,978 Parameters: domain_left_edge          = [ -512.  -512. -3584.]
yt : [INFO     ] 2018-10-09 01:32:54,980 Parameters: domain_right_edge         = [ 512.  512. 3584.]
yt : [INFO     ] 2018-10-09 01:32:54,981 Parameters: cosmologic

In [29]:
da = ds.all_data()
Nx, Ny, Nz = ds.domain_dimensions
iz = ds.domain_dimensions[2]/2 - 1
extent = (ds.domain_left_edge.v[0], ds.domain_right_edge.v[0],
          ds.domain_left_edge.v[2], ds.domain_right_edge.v[2])
nH = da['density'].v.reshape(Nx,Ny,Nz)
nHI = (da['density'].v*da['specific_scalar[3]'].v).reshape(Nx,Ny,Nz)
Erad0 = (da['rad_energy_density0'].v).reshape(Nx,Ny,Nz)*u.punit.value
Erad1 = (da['rad_energy_density1'].v).reshape(Nx,Ny,Nz)*u.punit.value
Erad0_pr = np.average(Erad0, axis=1)
Erad1_pr = np.average(Erad1, axis=1)

In [38]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import AxesGrid

NH = nH.sum(axis=1)
NHI = nHI.sum(axis=1)

fig = plt.figure(figsize=(20, 15))
grid = AxesGrid(fig, (0.075,0.075,0.85,0.85),
                nrows_ncols=(1,4), axes_pad=(0.5,0.1),
                label_mode="1", share_all=True, aspect=True,
                cbar_location="right", cbar_mode="each",
                cbar_size="4%", cbar_pad="2%")

data = (NH, nH[:,:,iz], Erad0_pr, Erad0[:,:,iz])
#        NHI, nHI[:,:,iz], Erad1_pr, Erad1[:,:,iz])
cmap = ('pink_r', 'Spectral_r', 'plasma', 'viridis')
#        'pink_r', 'Spectral_r', 'plasma', 'viridis')
label = (r'N_H [cm^-2]', r'n_H [cm^-3]',
         r'u_FUV (z-avg) [code unit]', r'u_FUV [code unit]')
#         r'N_H2 [cm^-2]', r'n_H2 [cm^-3]',
#         r'u_LW (z-avg) [code unit]', r'u_LW [code unit]')

im = []
for i, (data_, cmap_, label_) in enumerate(zip(data, cmap, label)):
    print label_,
    plt.sca(grid[i].axes)
    vmin = data_.min()
    vmax = data_.max()
    norm = mpl.colors.LogNorm()
    im_ = plt.imshow(data_.T, vmin=vmin, vmax=vmax, cmap=cmap_,
                     norm=norm, origin='lower', extent=extent)
    im.append(im_)
    cbar = plt.colorbar(im_, cax=grid.cbar_axes[i])
    cbar.set_label(label_)

 N_H [cm^-2] n_H [cm^-3] u_FUV (z-avg) [code unit]

ZeroDivisionError: float division by zero

Error in callback <function post_execute at 0x7f4c49b26668> (for post_execute):


ValueError: values must all be positive

ValueError: values must all be positive

<Figure size 1440x1080 with 8 Axes>

In [ ]:
da.